In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [34]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [35]:
df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)

In [36]:
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [37]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [38]:
one_hot_encoder_geography = OneHotEncoder()
geo_encoder = one_hot_encoder_geography.fit_transform(df[['Geography']])

In [39]:
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [40]:
one_hot_encoder_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [41]:
geo_df =pd.DataFrame(geo_encoder.toarray(), columns=one_hot_encoder_geography.get_feature_names_out(['Geography']))
geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [42]:
df = pd.concat([df.drop(columns=['Geography']),geo_df],axis=1)

In [43]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [44]:
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)

with open('one_hot_encoder_geography.pkl','wb') as f:
    pickle.dump(one_hot_encoder_geography,f)


In [45]:
X = df.drop(columns=['Exited'])
y = df['Exited']

In [46]:
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.2,random_state=42)

In [47]:
xtrain.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [48]:
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

In [49]:
xtrain

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [50]:
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

In [51]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(xtrain.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
    ])

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
#Another way to compile the model
#opt = tf.keras.optimizers.Adam(learning_rate=0.001)
#loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

In [54]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [55]:
logs_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = logs_dir, histogram_freq=1)

In [56]:
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [57]:
history = model.fit(
    xtrain,ytrain,validation_data=(xtest,ytest),epochs = 100, 
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4525 - accuracy: 0.8071 - val_loss: 0.3952 - val_accuracy: 0.8290
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3850 - accuracy: 0.8435 - val_loss: 0.3594 - val_accuracy: 0.8565
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3534 - accuracy: 0.8558 - val_loss: 0.3495 - val_accuracy: 0.8505
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3433 - accuracy: 0.8591 - val_loss: 0.3447 - val_accuracy: 0.8595
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3394 - accuracy: 0.8615 - val_loss: 0.3448 - val_accuracy: 0.8580
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3354 - accuracy: 0.8606 - val_loss: 0.3459 - val_accuracy: 0.8600
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3319 - accuracy: 0.8643 - val_loss: 0.3438 - val_accuracy: 0.8580

In [58]:
model.save('model.h5')

c:\Users\Abhis\OneDrive\Desktop\Projects\Churn Modelling\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [59]:
#%load_ext tensorboard

In [60]:
#%tensorboard --logdir logs/fit20250917-123146

In [61]:
X.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')